# Visualizing Chipotle's Data

This time we are going to pull data directly from the internet.
Special thanks to: https://github.com/justmarkham for sharing the dataset and materials.

### Step 1. Import the necessary libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

# set this so the graphs open internally
%matplotlib inline

In [2]:
import tensorflow as tf
print(f"TensorFlow version: {tf.__version__}")
print(f"GPUs disponibles: {len(tf.config.list_physical_devices('GPU'))}")
print(f"GPU support: {tf.test.is_gpu_available()}")

2025-07-01 18:53:23.965411: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-01 18:53:23.965446: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-01 18:53:23.966389: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-01 18:53:23.971026: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorFlow version: 2.15.0
GPUs disponibles: 1
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU support: True


2025-07-01 18:53:25.377609: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-01 18:53:25.398568: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-01 18:53:25.398780: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Test").getOrCreate()
print(f"PySpark version: {spark.version}")
spark.range(10).show()
spark.stop()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/01 18:53:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


PySpark version: 3.5.0
+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



In [4]:
import os
import sys

# Verificar variables de entorno importantes
env_vars = ['JAVA_HOME', 'PYSPARK_PYTHON', 'PYSPARK_DRIVER_PYTHON']
for var in env_vars:
    value = os.environ.get(var, 'No configurada')
    print(f"🔧 {var}: {value}")

print(f"🐍 Python version: {sys.version}")
print(f"📁 Working directory: {os.getcwd()}")


🔧 JAVA_HOME: /usr/lib/jvm/java-11-openjdk-amd64
🔧 PYSPARK_PYTHON: python3
🔧 PYSPARK_DRIVER_PYTHON: python3
🐍 Python version: 3.11.0rc1 (main, Aug 12 2022, 10:02:14) [GCC 11.2.0]
📁 Working directory: /tf


In [11]:

project_folder = "pyspark_exercises" 
if os.getcwd().endswith("/tf"):
    os.chdir(project_folder)


### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv). 

### Step 3. Assign it to a variable called chipo.

In [12]:
import os

os.getcwd()

'/tf/pyspark_exercises'

### Step 4. See the first 10 entries

### Step 5. Create a histogram of the top 5 items bought

### Step 6. Create a scatterplot with the number of items orderered per order price
#### Hint: Price should be in the X-axis and Items ordered in the Y-axis

### Step 7. BONUS: Create a question and a graph to answer your own question.